In [2]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import VGG16
from tqdm import tqdm

# Load and preprocess the images
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            image = img_to_array(load_img(img_path, target_size=(128, 128))) / 255.0
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

# Create pairs
def make_pairs(images, labels):
    pairs = []
    labels_pair = []
    num_classes = len(np.unique(labels))
    digit_indices = [np.where(labels == label)[0] for label in np.unique(labels)]

    for idx1 in range(len(images)):
        label = labels[idx1]
        same_class_indices = digit_indices[np.where(np.unique(labels) == label)[0][0]]
        idx2 = np.random.choice(same_class_indices)
        pairs.append([images[idx1], images[idx2]])
        labels_pair.append(1)

        diff_class_indices = np.concatenate([digit_indices[i] for i in range(num_classes) if i != np.where(np.unique(labels) == label)[0][0]])
        idx2 = np.random.choice(diff_class_indices)
        pairs.append([images[idx1], images[idx2]])
        labels_pair.append(0)

    return np.array(pairs), np.array(labels_pair)

# Load dataset
folder_path = "./converted_images"  # Update with your dataset path
images, labels = load_images_from_folder(folder_path)

# Create pairs
pairs, labels_pair = make_pairs(images, labels)

# Split the dataset into training, validation, and test sets
pairs_train, pairs_temp, labels_train, labels_temp = train_test_split(pairs, labels_pair, test_size=0.4, random_state=42)
pairs_val, pairs_test, labels_val, labels_test = train_test_split(pairs_temp, labels_temp, test_size=0.5, random_state=42)

# Build the Siamese network
def build_base_network(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)  # Reduced from 4096 to 1024
    return Model(inputs=base_model.input, outputs=x)

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

input_shape = (128, 128, 3)  # Reduced image size
base_network = build_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance)([processed_a, processed_b])

model = Model([input_a, input_b], distance)
model.compile(loss=contrastive_loss, optimizer=Adam(), metrics=['accuracy'])

# Train the model
history = model.fit([pairs_train[:, 0], pairs_train[:, 1]], labels_train, 
                    validation_data=([pairs_val[:, 0], pairs_val[:, 1]], labels_val),
                    batch_size=8,  # Reduced batch size
                    epochs=5)  # Reduced epochs

# Make predictions on the test set
y_pred = model.predict([pairs_test[:, 0], pairs_test[:, 1]])

# Convert distances to binary predictions
y_pred_binary = np.where(y_pred < 0.5, 1, 0)

# Calculate evaluation metrics
test_accuracy = accuracy_score(labels_test, y_pred_binary)
test_precision = precision_score(labels_test, y_pred_binary)
test_recall = recall_score(labels_test, y_pred_binary)
test_f1 = f1_score(labels_test, y_pred_binary)

print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Precision: {test_precision:.2f}")
print(f"Test Recall: {test_recall:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")


Epoch 1/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 234s 4s/step - accuracy: 0.3684 - loss: 1.4884 - val_accuracy: 0.4437 - val_loss: 1.8650
Epoch 2/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 244s 4s/step - accuracy: 0.2067 - loss: 0.2304 - val_accuracy: 0.1312 - val_loss: 0.0988
Epoch 3/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 253s 4s/step - accuracy: 0.1676 - loss: 0.1365 - val_accuracy: 0.1375 - val_loss: 0.0992
Epoch 4/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 275s 5s/step - accuracy: 0.1489 - loss: 0.1022 - val_accuracy: 0.1813 - val_loss: 0.1381
Epoch 5/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 224s 4s/step - accuracy: 0.2039 - loss: 0.2155 - val_accuracy: 0.2188 - val_loss: 0.1366
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step
Test Accuracy: 0.79
Test Precision: 0.82
Test Recall: 0.74
Test F1 Score: 0.78


In [4]:
pip install faiss-cpu



   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
    --------------------------------------- 0.3/14.6 MB ? eta -:--:--
   -- ------------------------------------- 1.0/14.6 MB 3.1 MB/s eta 0:00:05
   ----- ---------------------------------- 2.1/14.6 MB 5.3 MB/s eta 0:00:03
   --------- ------------------------------ 3.4/14.6 MB 5.0 MB/s eta 0:00:03
   ------------ --------------------------- 4.7/14.6 MB 5.2 MB/s eta 0:00:02
   --------------- ------------------------ 5.8/14.6 MB 5.3 MB/s eta 0:00:02
   ------------------- -------------------- 7.1/14.6 MB 5.3 MB/s eta 0:00:02
   ---------------------- ----------------- 8.1/14.6 MB 5.3 MB/s eta 0:00:02
   ------------------------- -------------- 9.4/14.6 MB 5.4 MB/s eta 0:00:01
   ---------------------------- ----------- 10.5/14.6 MB 5.4 MB/s eta 0:00:01
   ------------------------------ --------- 11.3/14.6 MB 5.2 MB/s eta 0:00:01
   --------------------------------- ------ 12.1/14.6 MB 5.1 MB/s eta 0:00:01
   ------

In [28]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import VGG16
import faiss
import tensorflow as tf

# Load and preprocess the images with reduced resolution
def load_images_from_folder(folder, target_size=(64, 64)):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            image = img_to_array(load_img(img_path, target_size=target_size)) / 255.0
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

# Build the Siamese network
def build_base_network(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)  # Reduced dense layer size
    return Model(inputs=base_model.input, outputs=x)

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

# Load dataset with reduced image size
folder_path = "./converted_images"  # Update with your dataset path
images, labels = load_images_from_folder(folder_path, target_size=(64, 64))

# Convert string labels to numeric labels
unique_labels = np.unique(labels)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
labels_numeric = np.array([label_to_index[label] for label in labels])

# Create pairs of images and labels (limited number of pairs)
def create_pairs(images, labels, num_pairs=5000):
    num_images = len(images)
    num_classes = len(np.unique(labels))
    pairs = []
    labels_pair = []
    
    for _ in range(num_pairs):
        # Randomly select two images
        idx1, idx2 = np.random.choice(num_images, 2, replace=False)
        img1, img2 = images[idx1], images[idx2]
        lbl1, lbl2 = labels[idx1], labels[idx2]
        
        pairs.append((img1, img2))
        labels_pair.append(1 if lbl1 == lbl2 else 0)
    
    return np.array(pairs), np.array(labels_pair)

# Create pairs and labels for training
pairs, labels_pair = create_pairs(images, labels_numeric)

# Split the dataset into training, validation, and test sets
pairs_train, pairs_temp, labels_train, labels_temp = train_test_split(pairs, labels_pair, test_size=0.4, random_state=42)
pairs_val, pairs_test, labels_val, labels_test = train_test_split(pairs_temp, labels_temp, test_size=0.5, random_state=42)

input_shape = (64, 64, 3)  # Adjusted to match image size
base_network = build_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance)([processed_a, processed_b])

model = Model([input_a, input_b], distance)
model.compile(loss=contrastive_loss, optimizer=Adam(), metrics=['accuracy'])

# Train the model
batch_size = 16
epochs = 5

history = model.fit([pairs_train[:, 0], pairs_train[:, 1]], labels_train.astype(np.float32), 
                    validation_data=([pairs_val[:, 0], pairs_val[:, 1]], labels_val.astype(np.float32)),
                    batch_size=batch_size,
                    epochs=epochs)

# Extract features for the dataset
def extract_features(images, model):
    return model.predict(images, batch_size=batch_size)

# Predict labels for test dataset
def predict_labels(test_images, database_images, database_labels, model):
    test_features = extract_features(test_images, model)
    database_features = extract_features(database_images, model)
    
    # Build the FAISS index
    d = test_features.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(database_features)
    
    # Search for nearest neighbors
    _, indices = index.search(test_features, 1)
    predicted_labels = [database_labels[i[0]] for i in indices]

    return np.array(predicted_labels)

# Create a database of images for label prediction
database_images = images
database_labels = labels_numeric

# Predict labels for the test dataset images
predicted_labels = predict_labels(pairs_test[:, 0], database_images, database_labels, base_network)

# Convert numeric labels back to original string labels
index_to_label = {index: label for label, index in label_to_index.items()}
labels_test_str = [index_to_label[label] for label in labels_test]
predicted_labels_str = [index_to_label[label] for label in predicted_labels]

# Calculate evaluation metrics
test_accuracy = accuracy_score(labels_test_str, predicted_labels_str)
test_precision = precision_score(labels_test_str, predicted_labels_str, average='weighted')
test_recall = recall_score(labels_test_str, predicted_labels_str, average='weighted')
test_f1 = f1_score(labels_test_str, predicted_labels_str, average='weighted')

print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Precision: {test_precision:.2f}")
print(f"Test Recall: {test_recall:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")


Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.0506 - loss: 54.4157 - val_accuracy: 0.1250 - val_loss: 0.0838
Epoch 2/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 309s 2s/step - accuracy: 0.1124 - loss: 0.6726 - val_accuracy: 0.0240 - val_loss: 2.9481
Epoch 3/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 311s 2s/step - accuracy: 0.1097 - loss: 0.9275 - val_accuracy: 0.0440 - val_loss: 0.6103
Epoch 4/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - accuracy: 0.1804 - loss: 3.7498 - val_accuracy: 0.9390 - val_loss: 0.6724
Epoch 5/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - accuracy: 0.8594 - loss: 1.5894 - val_accuracy: 0.9760 - val_loss: 0.9754
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 217ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 188ms/step
Test Accuracy: 0.98
Test Precision: 0.96
Test Recall: 0.98
Test F1 Score: 0.97


c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
